# What happens:
File is created and email is prepared

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = "C:/Users/WZHARBC/Documents/Python Monthly Tasks/Exports/RBI_TCVA"
os.chdir(directory)

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0002"

Please enter the RD (YYYYMMDD): 20221031


### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Save selection
### Also see how much time it took

In [4]:
%%time
d = pd.read_sql_query("select * from t_cva where booking_company = 'RBI'", connection_RAY)

Wall time: 3.22 s


### Save var in an excel file

In [5]:
# determining the name of the file
file_name = rd + '_T_CVA_RBI.xlsx'

#get last month
dt = datetime.datetime.today()
month_name = calendar.month_name[dt.month-1].upper()
year  = dt.year

# saving the excel
d.to_excel(file_name, index=False)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


### Send the email 

In [6]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'nikolina.zoroje-frimmel@rbinternational.com'
mail.Cc = 'nikolas.besemer@rbinternational.com; christian.gneist@rbinternational.com; markus.hochleitner@rbinternational.com; zoltan.szebenyi@rbinternational.com'
mail.Subject = 'T_CVA_RBI_' + rd
mail.Body = '''Dear colleagues,

Please find attached the current RAY data for ''' + month_name + " " + str(year) + '''

Kind regards,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''

# To attach a file to the email (optional):
attachment  = directory + "/" + file_name
mail.Attachments.Add(attachment)

mail.Display()